In [88]:
import gspread
import torch
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader    

In [89]:
credentials_path = '../riasec-model-1-c1119188e695.json' 

spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1x4e-yyXWHMZW9ZXiOb_oUnZLHukOWg1cc0Qf5jL2aLM/edit?usp=sharing'

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
gc = gspread.authorize(credentials)

spreadsheet = gc.open_by_url(spreadsheet_url)

worksheet = spreadsheet.get_worksheet(0)

data = worksheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

In [90]:
df

,Позначка часу,"When I receive gifts, I value practical and useful things more than something decorative.",My energy never runs out.,Theoretical ideas and dilemmas are not something worth spending a lot of time on.(How much do you agree with this statement),Do you like engaging in physical activity and sports?,Are you interested in working with various tools?,Do you like working outdoors?,Are you interested in solving complex mathematical problems?,Do you enjoy reading scientific books or journals?,I'm always interested to hear unique perspectives on things.,...,I am adept at persuading others to support my ideas.,I can easily change the minds of those around me regarding a certain topic.,Do you enjoy making important decisions and solving problems?,Do you participate in leadership or entrepreneurial initiatives?,Do you like order and systematic approach in work?,I always make maximum effort to avoid mistakes.,Do you find it easier to work with clear rules and established standards?,Are you interested in the organizational aspects of work?,I prefer following rules rather than improvising.,Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,3,2,4,3,5,4,3,5,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,2,3,1,4,3,5,3,4,2,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,3,4,5,5,4,4,3,5,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,1,1,5,1,1,5,1,1,1,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5,5,3,4,4,5,5,5,5,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,1,1,1,4,5,5,3,1,5,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,5,4,5,5,4,4,4,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,5,2,3,4,3,2,3,3,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,4,3,2,5,4,4,4,4,5,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4,2,4,3,2,4,2,3,2,...,2,3,4,3,4,3,4,3,4,3


In [91]:
df.columns

Index(['Позначка часу',
       'When I receive gifts, I value practical and useful things more than something decorative. ',
       'My energy never runs out. ',
       'Theoretical ideas and dilemmas are not something worth spending a lot of time on.(How much do you agree with this statement)',
       'Do you like engaging in physical activity and sports? ',
       'Are you interested in working with various tools? ',
       'Do you like working outdoors? ',
       'Are you interested in solving complex mathematical problems? ',
       'Do you enjoy reading scientific books or journals? ',
       'I'm always interested to hear unique perspectives on things. ',
       'I enjoy studying diverse information on any topic. ',
       'How do you feel about learning new technologies and programs? ',
       'People often tell me that my ideas are brilliant. ',
       'There is no form of art or event that I wouldn't want to explore/visit. ',
       'Space for expressing my creativity is impor

In [92]:
numeric_columns = df.columns[1:]

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

df

,Позначка часу,"When I receive gifts, I value practical and useful things more than something decorative.",My energy never runs out.,Theoretical ideas and dilemmas are not something worth spending a lot of time on.(How much do you agree with this statement),Do you like engaging in physical activity and sports?,Are you interested in working with various tools?,Do you like working outdoors?,Are you interested in solving complex mathematical problems?,Do you enjoy reading scientific books or journals?,I'm always interested to hear unique perspectives on things.,...,I am adept at persuading others to support my ideas.,I can easily change the minds of those around me regarding a certain topic.,Do you enjoy making important decisions and solving problems?,Do you participate in leadership or entrepreneurial initiatives?,Do you like order and systematic approach in work?,I always make maximum effort to avoid mistakes.,Do you find it easier to work with clear rules and established standards?,Are you interested in the organizational aspects of work?,I prefer following rules rather than improvising.,Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,3,2,4,3,5,4,3,5,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,2,3,1,4,3,5,3,4,2,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,3,4,5,5,4,4,3,5,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,1,1,5,1,1,5,1,1,1,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5,5,3,4,4,5,5,5,5,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,1,1,1,4,5,5,3,1,5,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,5,4,5,5,4,4,4,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,5,2,3,4,3,2,3,3,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,4,3,2,5,4,4,4,4,5,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4,2,4,3,2,4,2,3,2,...,2,3,4,3,4,3,4,3,4,3


In [93]:

numeric_columns = df.columns[1:]
riasec_columns = ['Realistic', 'Investigative', 'Artistic', 'Social', 'Enterprising', 'Conventional']
number_of_questions = 36
questions_per_section = 6
averages_df = pd.DataFrame()

for i in range(0, number_of_questions, questions_per_section):
    start_idx = i
    end_idx = i + questions_per_section
    selected_cols = df.columns[start_idx+1:end_idx]
    section_name = riasec_columns[int(end_idx/questions_per_section)-1]
    averages_df[section_name] = df[selected_cols].mean(axis=1) * (7/5)

final_df = pd.concat([df['Позначка часу'], averages_df], axis=1)
final_df_rm = final_df.where(final_df['Позначка часу'] == '13.12.2023 19:18:19')

final_df


,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04


In [94]:
df_merged = pd.merge(final_df, df, on='Позначка часу')

In [95]:
df_merged

,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional,"When I receive gifts, I value practical and useful things more than something decorative.",My energy never runs out.,Theoretical ideas and dilemmas are not something worth spending a lot of time on.(How much do you agree with this statement),...,I am adept at persuading others to support my ideas.,I can easily change the minds of those around me regarding a certain topic.,Do you enjoy making important decisions and solving problems?,Do you participate in leadership or entrepreneurial initiatives?,Do you like order and systematic approach in work?,I always make maximum effort to avoid mistakes.,Do you find it easier to work with clear rules and established standards?,Are you interested in the organizational aspects of work?,I prefer following rules rather than improvising.,Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32,3,2,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92,2,3,1,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32,3,4,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96,1,1,5,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76,5,5,3,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48,1,1,1,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32,5,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20,5,2,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60,4,3,2,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04,4,2,4,...,2,3,4,3,4,3,4,3,4,3


In [96]:
df_merged.fillna(0, inplace=True)

In [97]:
df_merged

,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional,"When I receive gifts, I value practical and useful things more than something decorative.",My energy never runs out.,Theoretical ideas and dilemmas are not something worth spending a lot of time on.(How much do you agree with this statement),...,I am adept at persuading others to support my ideas.,I can easily change the minds of those around me regarding a certain topic.,Do you enjoy making important decisions and solving problems?,Do you participate in leadership or entrepreneurial initiatives?,Do you like order and systematic approach in work?,I always make maximum effort to avoid mistakes.,Do you find it easier to work with clear rules and established standards?,Are you interested in the organizational aspects of work?,I prefer following rules rather than improvising.,Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32,3,2,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92,2,3,1,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32,3,4,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96,1,1,5,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76,5,5,3,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48,1,1,1,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32,5,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20,5,2,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60,4,3,2,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04,4,2,4,...,2,3,4,3,4,3,4,3,4,3


In [98]:
X = df_merged.iloc[0:1, 7:43].values 
y = df_merged.iloc[0:1, 1:7].values
X

array([[3., 2., 4., 3., 5., 4., 3., 5., 4., 3., 3., 4., 3., 2., 4., 3.,
        4., 2., 3., 2., 3., 2., 3., 4., 3., 2., 4., 3., 4., 3., 3., 4.,
        4., 3., 5., 3.]])

In [99]:
y

array([[4.76, 5.04, 4.48, 3.64, 4.48, 5.32]])

In [100]:
X = df_merged.iloc[:, 7:37].values  
y = df_merged.iloc[:, 1:7].values

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class FFNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = FFNModel(input_size=X.shape[1], output_size=y.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

num_epochs = 250
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
total_loss = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        print(outputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item()

average_loss = total_loss / len(test_loader)
print(f'Average Loss on Test Data: {average_loss}')

Epoch 1/250, Loss: 22.96632194519043
Epoch 2/250, Loss: 21.640003204345703
Epoch 3/250, Loss: 20.328174591064453
Epoch 4/250, Loss: 19.02987289428711
Epoch 5/250, Loss: 17.74668312072754
Epoch 6/250, Loss: 16.485353469848633
Epoch 7/250, Loss: 15.238995552062988
Epoch 8/250, Loss: 14.009725570678711
Epoch 9/250, Loss: 12.808717727661133
Epoch 10/250, Loss: 11.638899803161621
Epoch 11/250, Loss: 10.498540878295898
Epoch 12/250, Loss: 9.401248931884766
Epoch 13/250, Loss: 8.351688385009766
Epoch 14/250, Loss: 7.354092121124268
Epoch 15/250, Loss: 6.419987678527832
Epoch 16/250, Loss: 5.549625873565674
Epoch 17/250, Loss: 4.751373767852783
Epoch 18/250, Loss: 4.034485340118408
Epoch 19/250, Loss: 3.4031245708465576
Epoch 20/250, Loss: 2.8608622550964355
Epoch 21/250, Loss: 2.4061226844787598
Epoch 22/250, Loss: 2.0355052947998047
Epoch 23/250, Loss: 1.7439478635787964
Epoch 24/250, Loss: 1.5235310792922974
Epoch 25/250, Loss: 1.3654098510742188
Epoch 26/250, Loss: 1.2600682973861694
Epoch

In [101]:

df.fillna(0, inplace=True)



input_tensor = torch.tensor(X, dtype=torch.float32)

if len(input_tensor.shape) == 1:
    input_tensor = input_tensor.unsqueeze(0)

with torch.no_grad():
    predictions = model(input_tensor)

columns = ['R', 'I', 'A', 'S', 'E', 'C']
df_predictions = pd.DataFrame(predictions.numpy(), columns=columns)

df_predictions.insert(0, 'Позначка часу', df['Позначка часу'])
df_predictions.iloc[:, 1:] = df_predictions.iloc[:, 1:].applymap(lambda x: min(x, 7))
df_predictions

C:\Users\yg_ya\AppData\Local\Temp\ipykernel_19820\141191465.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_predictions.iloc[:, 1:] = df_predictions.iloc[:, 1:].applymap(lambda x: min(x, 7))


,Позначка часу,R,I,A,S,E,C
0,13.12.2023 17:14:56,4.770010,4.888801,3.919649,4.083486,4.493432,4.544231
1,13.12.2023 17:21:03,3.636275,3.875475,4.022616,3.644827,3.324501,3.685730
2,13.12.2023 17:47:13,5.691237,6.101438,6.444569,6.178538,5.798062,5.644948
3,13.12.2023 18:03:34,2.786595,1.786398,5.875972,2.468350,3.546525,2.147117
4,13.12.2023 18:05:36,5.441037,6.940969,5.082932,5.280063,6.570086,5.787684
5,13.12.2023 18:09:50,3.067230,4.380589,6.862977,3.948239,3.514117,4.125989
6,13.12.2023 18:42:14,6.529834,5.776596,4.198073,4.184861,5.015310,5.635009
7,13.12.2023 19:18:19,4.564214,5.045203,2.428350,3.792471,3.481907,4.390624
8,13.12.2023 21:09:46,4.767563,6.565760,5.859940,5.615718,6.273192,5.685851
9,16.12.2023 11:36:37,4.190808,4.205411,4.383381,4.391564,4.656640,4.361071


In [102]:
final_df

,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04


In [103]:
input_score = [1, 2, 3, 4, 5, 6]
given_score = [0, 0, 0, 0, 0, 0]

def calculate_average_similarity(input_score, given_score):
    differences = [abs(a - b) / max(a, b) for a, b in zip(input_score, given_score)]
    average_difference = sum(differences) / len(differences)
    return 1 - average_difference

result = calculate_average_similarity(input_score, given_score)
print("Average Similarity:", result)


Average Similarity: 0.0


In [104]:
def similarity(init_data, external_data, k=2):
    for i in range(1,4):
        if int(external_data[-i]) == 0:
            continue
        init_data[int(external_data[-i]-1)]*=k
        external_data[int(external_data[-i]-1)]*=k
    prod=0
    mod_1=0
    mod_2=0
    for i in range(6): # length of RIASEC
        prod+=init_data[i]*external_data[i]
        mod_1+=init_data[i]**2
        mod_2+=external_data[i]**2
    return float(prod)/((mod_1**0.5)*(mod_2**0.5))


In [105]:
#GETTING N (by defaul 20) top matches:

def get_top_results(data, n=100, value_index=1):
    top_results = sorted(data, key=lambda x: x[value_index], reverse=True)[:n]
    return top_results

In [108]:
riasec_columns = ['R', 'I', 'A', 'S', 'E', 'C']
subset_df = df_predictions[riasec_columns]
riasec_lists = [list(x) for x in subset_df.to_records(index=False)]

print(riasec_lists[len(riasec_lists)-1])

[4.0517173, 5.3877172, 2.5904934, 3.293403, 4.2884603, 4.7438135]


In [109]:
init_data = riasec_lists[len(riasec_lists)-2]

some_other_data_df = pd.read_excel('Interests.xlsx')

result = []

len(some_other_data_df)

for i in range (0, len(some_other_data_df ), 9):
    profession = some_other_data_df.loc[i, 'Title']

    R_score = some_other_data_df.loc[i, 'Data Value']
    I_score = some_other_data_df.loc[i+1, 'Data Value']
    A_score = some_other_data_df.loc[i+2, 'Data Value']
    S_score = some_other_data_df.loc[i+3, 'Data Value']
    E_score = some_other_data_df.loc[i+4, 'Data Value']
    C_score = some_other_data_df.loc[i+5, 'Data Value']
    HP_1 = some_other_data_df.loc[i+6, 'Data Value']
    HP_2 = some_other_data_df.loc[i+7, 'Data Value']
    HP_3 = some_other_data_df.loc[i+8, 'Data Value']

    row_values = (R_score, I_score, A_score, S_score, E_score, C_score, HP_1, HP_2, HP_3)



    tuple = (profession, similarity(list(init_data), list(row_values)))
    result.append(tuple)

print(get_top_results(result))

[('Ophthalmic Medical Technologists', 0.991830794338246), ('Computer Network Architects', 0.9832761095946616), ('Computer User Support Specialists', 0.9826328442562907), ('Instructional Designers and Technologists', 0.9815471623674367), ('Industrial Safety and Health Engineers', 0.9801829374437283), ('Database Architects', 0.9801255558020631), ('Intelligence Analysts', 0.9800844993940141), ('Anesthesiologists', 0.9798021519725562), ('Diagnostic Medical Sonographers', 0.9780918081721549), ('Operations Research Analysts', 0.9780650516744518), ('Fire-Prevention and Protection Engineers', 0.9775211780139441), ('Web Developers', 0.9773458736154438), ('Surgeons', 0.9772369621465794), ('Internists, General', 0.9770802341749952), ('Ophthalmic Medical Technicians', 0.9766079505412018), ('Forest and Conservation Technicians', 0.9765620744593758), ('Cardiovascular Technologists and Technicians', 0.9763433704119255), ('Informatics Nurse Specialists', 0.9761619003247282), ('Fire Investigators', 0.9

In [110]:
for job, probability in get_top_results(result):
    print(f"{job} : {probability * 100}%")

Ophthalmic Medical Technologists : 99.18307943382459%
Computer Network Architects : 98.32761095946616%
Computer User Support Specialists : 98.26328442562907%
Instructional Designers and Technologists : 98.15471623674367%
Industrial Safety and Health Engineers : 98.01829374437283%
Database Architects : 98.01255558020631%
Intelligence Analysts : 98.00844993940142%
Anesthesiologists : 97.98021519725562%
Diagnostic Medical Sonographers : 97.80918081721549%
Operations Research Analysts : 97.80650516744518%
Fire-Prevention and Protection Engineers : 97.7521178013944%
Web Developers : 97.73458736154438%
Surgeons : 97.72369621465793%
Internists, General : 97.70802341749952%
Ophthalmic Medical Technicians : 97.66079505412017%
Forest and Conservation Technicians : 97.65620744593758%
Cardiovascular Technologists and Technicians : 97.63433704119255%
Informatics Nurse Specialists : 97.61619003247282%
Fire Investigators : 97.6069938637501%
Magnetic Resonance Imaging Technologists : 97.58209956840949